In [1]:
import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import AutoTokenizer

from torch.utils.data import DataLoader, Dataset


# Load the dataset
dataset = load_dataset("emotion")

# Split train/test
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Preprocess the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Create a custom dataset
class EmotionDataset(Dataset):
    """Custom Dataset for Emotion Classification

    A PyTorch Dataset class that handles emotion text data and their corresponding labels.
    It wraps the tokenized text encodings and emotion labels for use with DataLoader.

    Args:
        encodings (BatchEncoding): The tokenized text encodings from a BERT tokenizer
        labels (list): List of emotion labels corresponding to the text samples

    Attributes:
        encodings (BatchEncoding): Stored tokenized text encodings
        labels (Tensor): Emotion labels converted to torch tensor
    """

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, idx):
        """Get a single sample from the dataset.

        Args:
            idx (int): Index of the sample to retrieve

        Returns:
            tuple: A dictionary containing the encoded text and its corresponding label
        """
        return {key: val[idx] for key, val in self.encodings.items()}, self.labels[idx]

    def __len__(self):
        """Get the total number of samples in the dataset.

        Returns:
            int: The number of samples in the dataset
        """
        return len(self.labels)


train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

In [4]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Define the model
class EmotionClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        """Initialize the Emotion Classifier model.

        A simple neural network architecture for emotion classification using word embeddings
        and 1D convolution. The model consists of an embedding layer, followed by a 1D 
        convolution, ReLU activation, global average pooling, and a final linear layer.

        Args:
            vocab_size (int): Size of the vocabulary from the tokenizer
            embedding_dim (int): Dimension of the word embeddings
            num_classes (int): Number of emotion classes to predict

        Attributes:
            embedding (nn.Embedding): Word embedding layer
            conv1d (nn.Conv1d): 1D convolutional layer
            relu (nn.ReLU): ReLU activation function
            global_avg_pool (nn.AdaptiveAvgPool1d): Global average pooling layer
            fc (nn.Linear): Final linear layer for classification
        """
        super(EmotionClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d = nn.Conv1d(in_channels=embedding_dim,
                                out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        """Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of token IDs with shape (batch_size, sequence_length)

        Returns:
            torch.Tensor: Logits for each emotion class with shape (batch_size, num_classes)

        The forward pass performs the following operations:
        1. Convert token IDs to embeddings
        2. Permute dimensions for conv1d operation
        3. Apply 1D convolution
        4. Apply ReLU activation
        5. Apply global average pooling
        6. Squeeze unnecessary dimension
        7. Apply final linear layer
        """
        x = self.embedding(x)           # (batch_size, seq_len, embedding_dim)
        x = x.permute(0, 2, 1)         # (batch_size, embedding_dim, seq_len)
        x = self.conv1d(x)             # (batch_size, 64, seq_len)
        x = self.relu(x)               # (batch_size, 64, seq_len)
        x = self.global_avg_pool(x)    # (batch_size, 64, 1)
        x.squeeze_(2)                  # (batch_size, 64)
        x = self.fc(x)                 # (batch_size, num_classes)
        return x

In [7]:
# Initialize the model
vocab_size = tokenizer.vocab_size
embedding_dim = 128
num_classes = len(set(train_labels))
model = EmotionClassifier(vocab_size, embedding_dim, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch, labels in train_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(batch['input_ids'])
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(
        f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1/5, Loss: 1.4640
Epoch 2/5, Loss: 0.7608
Epoch 3/5, Loss: 0.3618
Epoch 4/5, Loss: 0.2132
Epoch 5/5, Loss: 0.1414


In [ ]:
def predict(sentence):
    """Predicts the emotion for a given text sentence using the trained EmotionClassifier model.

    Args:
        sentence (str): The input text sentence for emotion prediction.

    Returns:
        str: The predicted emotion label from the set ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'].
             Returns "Unknown emotion" if the predicted index is out of range.

    The function performs the following steps:
    1. Sets model to evaluation mode
    2. Tokenizes the input sentence
    3. Moves tensors to the appropriate device
    4. Makes prediction using the model
    5. Maps the predicted class index to emotion label
    """
    model.eval()
    encoded = tokenize([sentence])
    encoded = {key: val.to(device) for key, val in encoded.items()}

    with torch.no_grad():
        output = model(encoded['input_ids'])
        predicted = torch.argmax(output, dim=1).cpu().item()

    # Correct emotion mapping based on the "emotion" dataset labels
    emotions = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

    try:
        return emotions[predicted]
    except IndexError:
        return "Unknown emotion"


# Test the prediction
sample_sentence = "I am so excited about this new adventure!"
print(f"Input: {sample_sentence}")
print(f"Predicted Emotion: {predict(sample_sentence)}")

Input: I am so excited about this new adventure!
Predicted Emotion: joy
